# Example of generation 

In [1]:
import torch
from config import get_config
from model import CDGPT
from tokenizer import SentencePieceTokenizer


In [5]:
tokenizer_path = "checkpoints/tokenizer.model"
tokenizer = SentencePieceTokenizer(tokenizer_path)
tokenizer.vocab.values()


dict_values(['<unk>', '<s>', '</s>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'B', 'J', 'O', 'U', 'X', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\t', '\n', '\r', '\x0b', '\x0c', 'Α', 'Β', 'Γ', 'Δ', 'Ε', 'Ζ', 'Η', 'Θ', 'Ι', 'Κ', 'Λ', 'Μ', 'Ν', 'Ξ', 'Ο', 'Π', 'Ρ', '\u03a2', 'Σ', 'Τ', 'Υ', 'Φ', 'Χ', 'Ψ', 'Ω', 'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ξ', 'ο', 'π', 'ρ', 'ς', 'σ', 'τ', 'υ', 'φ', 'χ', 'ψ', 'ω', '<CLS>', 'SEP', '<PAD>', '<MASK>', '<EOD>', '<:>', '£', '¥', '«', '»', '±', '·', '<R93>', '<R94>', '<R95>', '<R96>', '<R97>', '<R98>', '<R99>', '<R100>', '<R101>', '<R102>', '<R103>', '<R104>', '<R105>', '<R106>', '<R107>', '<R108>', '<R109>', '<R110>', '<R111>', '<R112>', '<R113>', '<R114>', '<R1

In [2]:
torch.set_grad_enabled(False)

In [4]:
tokenizer_path = "checkpoints/tokenizer.model"
cfg = get_config()
cfg.tokenizer.path = tokenizer_path
tokenizer = SentencePieceTokenizer(tokenizer_path)

In [5]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

fasta_file = "example.fasta"
parser = SeqIO.parse(fasta_file, "fasta")
dna_record = next(parser)
protein_record = next(parser)
reverse_translate_record = next(parser)

## Translation generation

In [6]:
model_path = "checkpoints/CD-GPT-1b.pth"
state = torch.load(model_path, map_location="cpu")
model = CDGPT(cfg)
model.load_state_dict(state["model"], strict=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.half().to(device).eval()

number of parameters: 1059.38M


CDGPT(
  (transformer): ModuleDict(
    (wte): Embedding(64000, 2304)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=2304, out_features=6912, bias=False)
          (c_proj): Linear(in_features=2304, out_features=2304, bias=False)
        )
        (mlp): SwiGLU(
          (c_fc1): Linear(in_features=2304, out_features=6144, bias=False)
          (c_fc2): Linear(in_features=2304, out_features=6144, bias=False)
          (c_proj): Linear(in_features=6144, out_features=2304, bias=False)
        )
        (rms_1): RMSNorm()
        (rms_2): RMSNorm()
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2304, out_features=64000, bias=False)
)

In [8]:
prompt = f"<mRNA>{str(dna_record.seq)}</mRNA><translate><:>"
x = tokenizer.encode(prompt, eos=False, device=device)
output = model.generate(x,
                        max_new_tokens=128,
                        temperature=0.8,
                        top_k=128,
                        top_p=0.0,
                        stop_ids=(tokenizer.bos, tokenizer.eos, tokenizer.pad)
                        )
output = tokenizer.decode(output.sequences)

In [9]:
output = output[len(prompt):]
translate_res = output.split("</protein>")[0]

In [10]:
translate_gt = str(dna_record.seq.translate())
print(f"GROUND TRUTH ABOVE, GENERATION BELOW, MISMATCHES IN \033[91mRED\033[0m")
print(translate_gt)
for i in range(len(translate_gt)):
    if translate_res[i] == translate_gt[i]:
        print(translate_res[i], end="")
    else:
        print(f"\033[91m{translate_res[i]}\033[0m", end="")


GROUND TRUTH ABOVE, GENERATION BELOW, MISMATCHES IN RED
MSGLHRSSSAPMKKGLPPKELLDDLCSRFVLNVPKEDLESFERILFLVENAHWFYEDNSVERDPSLKSLTLKEFTSLIFNSCDVLKPYVPHLDDIFKDFTSYKVRVPVTGAIIL
MSGLHRSSSAAPMKKGLPPKELLDDLCSRFVLNVPKEDLESFERILFLVENAHWFYEDNSVERDPSLKSLTLKEFTSLIFNCCDVLKPYVPHLDDIFKDFTSYKVRVPVTGAII

## Reverse translation generation

In [11]:
# you can download this model from Tencent Weiyun Disk.
model_path = "checkpoints/CD-GPT-1b-reverse-translation.pth"
state = torch.load(model_path, map_location="cpu")
model = CDGPT(cfg)
model.load_state_dict(state["model"], strict=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.half().to(device).eval()

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/CD-GPT-1b-reverse-translation.pth'

In [6]:
prompt = f"<protein>{str(protein_record.seq)}</protein><reverse_translate><:>"
x = tokenizer.encode(prompt, eos=False, device=device)
output = model.generate(x,
                        max_new_tokens=1024,
                        temperature=0.8,
                        top_k=128,
                        top_p=0.0,
                        stop_ids=(tokenizer.bos, tokenizer.eos, tokenizer.pad)
                        )
output = tokenizer.decode(output.sequences)

In [7]:
output = output[len(prompt):]
reverse_translate_res = output.split("</mRNA>")[0].split("<mRNA>")[-1]

In [ ]:
reverse_translate_gt = str(reverse_translate_record.seq)
print(f"GROUND TRUTH ABOVE, GENERATION BELOW, MISMATCHES IN \033[91mRED\033[0m")
print(reverse_translate_gt)
for i in range(len(reverse_translate_gt)):
    if i >= len(reverse_translate_res):
        break
    if reverse_translate_res[i] == reverse_translate_gt[i]:
        print(reverse_translate_res[i], end="")
    else:
        print(f"\033[91m{reverse_translate_res[i]}\033[0m", end="")